### Since we're running everything on Google's remote servers, we need to re-load all the software packages we'll use each time we open a notebook.

### Please run the following two cells to install all the necessary software. Note that it could take a couple of minutes.

In [ ]:
#STEP 1: Enable the "Anaconda" package manager in this Google Colab Notebook:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#STEP 2: Install the free SCIP optimization problem solver
!conda install conda-forge::scip

#STEP 3: Download and install my "PyomoTools" package.
#   This will install all other pieces of software we need.
!git clone https://github.com/NathanDavisBarrett/PyomoTools.git
%cd PyomoTools
!pip install .
%cd ..

#At the end of this cell, you'll get a popup window asking you to restart. Please click "cancel".

# Two-Stage Retirement Planning Model

$$max \sum_{s \in \textbf{S}} \pi_s \sum_{a \in \textbf{A}} B_{a,t^{END},s}$$
$$----\text{ subject to }----$$
$$B_{a,t,s} = \left(B_{a,t-1,s}-W_{a,t-1,s}\right) \times (1 + \delta_{a,s}) \ \ \forall a \in \textbf{A}, t \in \textbf{T}^{NON-INIT}, s \in \textbf{S}$$
$$\alpha_a W_{a,t,s} = \omega_t F_a \ \ \forall a \in \textbf{A}, t \in \textbf{T}, s \in \textbf{S}, $$
$$B_{a,0,s} = \beta{a} \ \ \forall a \in \textbf{A}, s \in \textbf{S}$$
$$W_{a,t,s} \leq 0 \ \ \forall a \in \textbf{A}, t \in \textbf{T}, s \in \textbf{S}$$
$$\sum_{a \in \textbf{A}} F_a = 1$$

In [102]:
import pyomo.environ as pyo
import numpy as np
import matplotlib.pyplot as plt

### STEP 1: Define Sets

In [103]:
model = pyo.ConcreteModel()

model.setA = pyo.Set(initialize=["IRA", "401k", "Brokerage"])
model.setT = pyo.Set(initialize=list(range(12 * 25))) #12 months over 25 years
model.setS = pyo.Set(initialize=[1,2,3,4])

### STEP 2: Define Parameters

In [104]:
alpha = {
    "IRA": 1,
    "401k": 0.8,
    "Brokerage": 0.7
}

beta = {
    "IRA":       300000,
    "401k":      900000,
    "Brokerage":  50000 
}

pi = {1: 0.05, 2: 0.1, 3: 0.6, 4: 0.25}

delta = {
    "IRA": {
        1: 0.02/12,
        2: 0.07/12,
        3: 0.07/12,
        4: 0.08/12
    },
    "401k": {
        1: 0.03/12,
        2: 0.06/12,
        3: 0.10/12,
        4: 0.05/12
    },
    "Brokerage": {
        1: -0.02/12,
        2: 0.20/12,
        3: 0.12/12,
        4: 0.00/12
    }
}

omega = {t: 8000*(1+0.02/12)**t for t in model.setT} #"**" means exponent in python

### STEP 3: Define Variables

In [105]:
model.B = pyo.Var(model.setA * model.setT * model.setS, domain=pyo.NonNegativeReals)
model.W = pyo.Var(model.setA * model.setT * model.setS, domain=pyo.NonNegativeReals)

model.F = pyo.Var(model.setA,domain=pyo.NonNegativeReals)


### STEP 4: Define Constraints

In [106]:
model.setT_NON_INIT = pyo.Set(initialize=[t for t in model.setT if t != 0])

def SubsequentBalanceEquation(model,a,t,s):
    return model.B[a,t,s] == (model.B[a,t-1,s]-model.W[a,t-1,s]) * (1 + delta[a][s])
model.SubsequentBalanceEquation = pyo.Constraint(model.setA * model.setT_NON_INIT * model.setS, rule=SubsequentBalanceEquation)

def WithdrawalEnforcement(model,a,t,s):
    return alpha[a] * model.W[a,t,s] == model.F[a] * omega[t]
model.WithdrawalEnforcement = pyo.Constraint(model.setA * model.setT * model.setS,rule=WithdrawalEnforcement)

def InitialBalanceEnforcement(model,a,s):
    return model.B[a,0,s] == beta[a]
model.InitialBalanceEnforcement = pyo.Constraint(model.setA * model.setS,rule=InitialBalanceEnforcement)

def Sum_F_Must_Be_1(model):
    return sum(model.F[a] for a in model.setA) == 1
model.Sum_F_Must_Be_1 = pyo.Constraint(rule=Sum_F_Must_Be_1)

### STEP 5: Define Objective

In [107]:
tEnd = np.max(model.setT)

model.Obj = pyo.Objective(expr=sum(pi[s] * model.B[a,tEnd,s] for a in model.setA for s in model.setS), sense=pyo.maximize)

### STEP 6: Solve the Model

In [108]:
solver = pyo.SolverFactory("scip")
solver.solve(model);

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: infeasible


### STEP 7: Extract the Results

For our stochastic model, the real result here is what **policy** we should adopt. Then, when we're going through real life (a scenario that probably did't exactly appear in our scenarios), we'll just use this generic policy.

For us, the policy was to find which fraction to withdraw from each account: $F_{a}$

In [109]:
for a in model.setA:
    print(f"F[{a}] = {pyo.value(model.F[a]):.2f}")

ERROR: evaluating object as numeric value: F[IRA]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object F[IRA]


ValueError: No value for uninitialized NumericValue object F[IRA]

### AH! The model is infeasible! Why might that be? 

* You might not have enough money in your accounts to that your ending balance is not negative for EVERY scenario.
* How can we rectify this?